<a href="https://colab.research.google.com/github/zcwisc/GB657/blob/main/Module_1_Clustering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Unsupervised Learning: Clustering

As mentioned, there are (at least) two basic machine learning setups.  In **supervised** machine learning one observes a response $Y$ with observing $p$ different features $X=(X_1,X_2,\ldots,X_p)$, where we typically postulate the relationship $Y = f(X)+\varepsilon$ and $\varepsilon$ independent of $X$ with mean zero. Here quality is usually assessed by the (test/out-of-sample) error that compares predictions and realizations for a separate dataset.  

In **unsupervised** learning, we only observe $p$ features $X_1,X_2,\ldots,X_p$, and we would like to learn about their relationship -- without focussing on a supervising outcome.  Of course, the difficulty is how to assess quality in this case -- so different unsupervised learning techniques are very different, and which one to pick will depend on the nature of the problem.

In this tutorial, we will take a closer look at one important class of unsupervised learning algorithms: **Cluster Analysis**.  We start with a quick introduction but then look into the applications of these techniques in more detail in the context of a case study.

As usual, we start by implementing the relevant packages:

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.lines as mlines
import plotly.figure_factory as ff
import pandas as pd
import seaborn as sns
from random import sample

from sklearn.preprocessing import MinMaxScaler, StandardScaler # For rescaling metrics to fit 0 to 1 range
from sklearn.metrics import multilabel_confusion_matrix, confusion_matrix
from sklearn.cluster import KMeans, AgglomerativeClustering
from scipy.cluster.hierarchy import dendrogram, linkage
from scipy.spatial.distance import euclidean

## Clustering Analysis Background

### Background

*Clustering* refers to techniques for finding subgroups in a given dataset. The typical approach to determine clusters $C_1,\ldots,C_K$ is to minimize:
$$
\sum_{k=1}^K W(C_k),
$$
where $W$ is a measure of *variation* within a cluster.  For instance, **k-means clustering** uses the Euclidean distance to measure variation:
$$
W(C_k) = \frac{1}{|C_k|} \sum_{i,i' \in C_k} \sum_{j=1}^p (x_{ij} - x_{i'j})^2.
$$
The algorithms are implemented via a greedy algorithm by considering the centers of clusters (referred to as *centroids*).  The number of clusters $K$ must be chosen beforehand.  One approach is *hierarchical clustering*, where one starts with a larger number of clusters and then *fuses* custers that are similar (e.g., with regards to the distance between their centroids).

### Simulated Example

Let's consider a very basic simulated example -- let's simulate normal random variables with different means:

In [ ]:
X_raw = np.random.multivariate_normal((0,0), np.array([[1, 0], [0, 1]]), size=100)
X_raw[0:49,0]=X_raw[1:50,0]+3
X_raw[0:49,1]=X_raw[1:50,1]-4

Let's plot:

In [ ]:
plt.figure(figsize = (6,4))
plt.scatter(X_raw[0:49,0], X_raw[0:49,1], color='red')
plt.scatter(X_raw[50:99,0], X_raw[50:99,1], color='black')
plt.xlabel('X0')
plt.ylabel('X1')
plt.legend()
plt.show()

And let's run k means clustering with two clusters (notice that we are setting a random state so we can replicate the process):


In [ ]:
kmeans = KMeans(n_clusters = 2, init = 'k-means++', max_iter = 1000, random_state = 123)
kmeans.fit(X_raw)
centroids = kmeans.cluster_centers_
centroids

Let's include the centroids in the picture:

In [ ]:
plt.figure(figsize = (6,4))
plt.scatter(X_raw[0:49,0], X_raw[0:49,1], color='red', label='Cluster 1')
plt.scatter(X_raw[50:99,0], X_raw[50:99,1], color='black', label='Cluster 2')
plt.scatter(centroids[:,0], centroids[:,1], color='blue', marker='X', s=200, label='Centroids') # Plot centroids
plt.xlabel('X0')
plt.ylabel('X1')
plt.legend()
plt.show()

And let's check the cluster allocation:

In [ ]:
label = kmeans.fit_predict(X_raw)
label

In [ ]:
plt.figure(figsize = (6,4))
plt.scatter(X_raw[:, 0], X_raw[:, 1], c=label, cmap='viridis')
plt.xlabel('X0')
plt.ylabel('X1')
plt.title('K-Means Clustering Results (Simulated Data)')
plt.show()

So the algorithm was able to identify how we set up the data!

## Cluster Analysis Case Study: County Health Rankings 2013

We analyze County Health Rankings in the US in 2013, based on a data set from the University of Wisconsin Population Health Institute.

### Data Preparation

Let's load the data:

In [ ]:
!git clone https://github.com/zcwisc/GB657

In [ ]:
health = pd.read_csv('GB657/Module_1_CountyHealthR.csv')

In [ ]:
health.info()

So it's a large dataset, and the first three numbers are indices.  However, the remaning columns provide rather detailed information for each county.  

Let's drop the features with many missing values and the index information. Afterwards, let's drop the counties with missing features.

In [ ]:
health = health.drop(columns=['FIPS','State','County','Perc.Fair.Poor.Health','Perc.Smokers','Perc.Excessive.Drinking','MV.Mortality.Rate','Pr.Care.Physician.Ratio','Perc.No.Soc.Emo.Support'])
health = health.dropna()

Let's look at the resulting data:

In [ ]:
health.info()

In [ ]:
health.describe()

Let's visualize:

In [ ]:
sns.heatmap(health.corr());

So quite a bit of correlation, but nothing seems to be "replicated."

And let's scale the data so as to make sure we are comparing apples to apples:

In [ ]:
scaler = MinMaxScaler()
scaler.fit(health)
health_sc = scaler.transform(health)
health_sc

### K-Means clustering

Let's commence with k-means clustering, which is avalable in the library `sklearn.cluster` (along with other clustering algorithms, see below). To decide on the number of clusters, we evaluate how the within-sum-of-squares varies between the number of clusters using a so-called *elbow plot*:


In [ ]:
wcss = []
for i in range(2, 12):
    kmeans = KMeans(n_clusters=i, init='k-means++', max_iter=1000, n_init=10, random_state=0)
    kmeans.fit(health_sc)
    wcss.append(kmeans.inertia_)
plt.bar(range(2, 12), wcss)
plt.title('Elbow Method')
plt.xlabel('Number of clusters')
plt.ylabel('WCSS')
plt.show()

It appears that six clusters seem to present a reasonable choice, there is a bit of an "elbow" there. So let's go with six:

In [ ]:
kmeans = KMeans(n_clusters=6, init='k-means++', max_iter=1000, n_init=10, random_state=0)
kmeans.fit(health_sc)

We can get the labels via:

In [ ]:
kmeans.labels_

Let's add to our dataset:

In [ ]:
health['k_means_cl'] = kmeans.labels_

Let's compare the different clusters by evaluating the YPLL.Rate (Years of Potential Life Lost).

In [ ]:
health[health['k_means_cl'] == 0]['YPLL.Rate'].mean()

In [ ]:
health[health['k_means_cl'] == 1]['YPLL.Rate'].mean()

In [ ]:
health[health['k_means_cl'] == 2]['YPLL.Rate'].mean()

In [ ]:
health[health['k_means_cl'] == 3]['YPLL.Rate'].mean()

In [ ]:
health[health['k_means_cl'] == 4]['YPLL.Rate'].mean()

In [ ]:
health[health['k_means_cl'] == 5]['YPLL.Rate'].mean()

Let's relabel so it's increasing:

In [ ]:
def relab(label):
  if label == 0:
    return 0
  elif label == 1:
    return 1
  elif label == 2:
    return 5
  elif label == 3:
    return 4
  elif label == 4:
    return 3
  else:
    return 2

In [ ]:
kmlab = list(map(relab, kmeans.labels_))

In [ ]:
health['k_means_cl'] = kmlab

### Hierarchical Clustering

Let's use the hierarchical clustering algorithm in scikit (`AgglomerativeClustering`). Plotting a *dendrogram* is easier with the `scipy.cluster.hierarchy`, yet given the size of the dataset we don't see a ton here.

To compare with the k-means approach, let's run a hierarchical clustering also with six clusters.

In [ ]:
hierarch = AgglomerativeClustering(n_clusters=6)
hierarch.fit(health_sc)

Again, we can get the labels via:

In [ ]:
hierarch.labels_

Let's also add to the dataset and carry out a similar approach as before:

In [ ]:
health['k_hier_cl'] = hierarch.labels_

In [ ]:
health[health['k_hier_cl'] == 0]['YPLL.Rate'].mean()

In [ ]:
health[health['k_hier_cl'] == 1]['YPLL.Rate'].mean()

In [ ]:
health[health['k_hier_cl'] == 2]['YPLL.Rate'].mean()

In [ ]:
health[health['k_hier_cl'] == 3]['YPLL.Rate'].mean()

In [ ]:
health[health['k_hier_cl'] == 4]['YPLL.Rate'].mean()

In [ ]:
health[health['k_hier_cl'] == 5]['YPLL.Rate'].mean()

In [ ]:
def relab2(label):
  if label == 0:
    return 5
  elif label == 1:
    return 2
  elif label == 2:
    return 0
  elif label == 3:
    return 4
  elif label == 4:
    return 3
  else:
    return 1

In [ ]:
hierlab = list(map(relab2, hierarch.labels_))

In [ ]:
health['k_hier_cl'] = hierlab

Let's compare the two:

In [ ]:
labels = [0,1,2,3,4,5]
confusion_matrix(health['k_means_cl'],health['k_hier_cl'],labels=labels)

So it looks like there is quite a bit of overlap, particularly with regards to clusters 0, 1, 2, and 5. It appears clusters 3 and 4 are overlapping a little bit. However, it also is clear that assigning points to clusters isn't highly obvious.

### Visualizing Clusters

Let's visualize clusters by comparing the distributions of some of the features across clusters. As we had discussed, comparing distributions can be conveniently accomplished via box-whisker plots.

Let's start with children in poverty.

In [ ]:
sns.boxplot(x = "k_means_cl", y = "Perc.Children.in.Poverty", data = health)
plt.show()

Recall that we sorted the clusters according to YPLL.Rate (Years of Potential Life Lost), so more years of life lost is a sign of worse health. It appears that this weakly aligns with children poverty.

Let's look at violent crimes.

In [ ]:
sns.boxplot(x = "k_means_cl", y = "Violent.Crime.Rate", data = health)
plt.show()

So here we have a weak alignment, although cluster 4 seems a bit lower. Let's look at fast food concentration.

In [ ]:
sns.boxplot(x = "k_means_cl", y = "Perc.Fast.Foods", data = health)
plt.show()

Let's check particulates:

In [ ]:
sns.boxplot(x = "k_means_cl", y = "Avg.Daily.Particulates", data = health)
plt.show()

So the counties in cluster 1 and 4 seem to have more pollution.

We can look more into the clusters and try to describe them. One way to think about getting a good interpretation is to think of **stereotypes** of each clusters. That is, thinking about the "typical stick-figure" representative of cluster representatives: Imagine the typical example of a county in this cluster.

However, here, rather than looking more into the clusters, what we will do is to plot a map with US counties and color them according to their cluster allocations. The code is a little complex (I used chatpgt to generate it :-) but it works.

First, let's make a dataset that has FIPS (county) codes and the cluster allocations:

In [ ]:
health2 = pd.read_csv('GB657/Module_1_CountyHealthR.csv')
fips_codes = health2['FIPS']
cluster_labels = health['k_means_cl']

In [ ]:
fips_cluster_df = pd.DataFrame({'fips': fips_codes, 'cluster': cluster_labels})

And, then, let's plot the map:

In [ ]:
import geopandas as gpd
import pandas as pd
import matplotlib.colors as mcolors
import requests
from io import BytesIO
from zipfile import ZipFile

# Step 1: Download TIGER/Line U.S. counties shapefile (using 2023 data)
url = "https://www2.census.gov/geo/tiger/TIGER2023/COUNTY/tl_2023_us_county.zip"
response = requests.get(url)

# Extract the zipfile into memory
with ZipFile(BytesIO(response.content)) as zf:
    zf.extractall("tl_2023_us_county")  # Extract to a folder

# Load the shapefile
counties = gpd.read_file("tl_2023_us_county/tl_2023_us_county.shp")

# Ensure FIPS codes are strings and zero-padded to 5 characters
fips_cluster_df['fips'] = fips_cluster_df['fips'].astype(str).str.zfill(5)

# Step 2: Merge the shapefile data with the cluster data
counties = counties.merge(fips_cluster_df, left_on='GEOID', right_on='fips', how='left')

# Step 3: Define a custom colormap with only as many colors as clusters
unique_clusters = counties['cluster'].dropna().unique()
num_clusters = len(unique_clusters)

# Use a discrete colormap for clarity
# cmap = plt.cm.get_cmap('tab20', num_clusters)  # Get only `num_clusters` colors
# norm = mcolors.BoundaryNorm(range(num_clusters + 1), cmap.N)

# Step 4: Plot the map with a larger size
fig, ax = plt.subplots(1, 1, figsize=(16, 12))

# Create discrete colormap (viridis style, but one color per cluster)
unique_clusters = sorted(counties['cluster'].dropna().unique())
num_clusters = len(unique_clusters)
cmap = plt.cm.get_cmap('viridis', num_clusters)
norm = mcolors.BoundaryNorm(np.arange(num_clusters + 1) - 0.5, num_clusters)

# Plot without GeoPandas legend
counties.plot(column='cluster', cmap=cmap, ax=ax, legend=False,
              norm=norm, missing_kwds={"color": "lightgrey", "label": "No Data"})

# Add compact colorbar with discrete ticks
sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
sm.set_array([])
cbar = plt.colorbar(sm, ax=ax, fraction=0.03, pad=0.02,
                    ticks=np.arange(num_clusters))
cbar.ax.tick_params(labelsize=8)
cbar.set_label("Cluster", fontsize=10)

# Zoom and clean up
ax.set_xlim(-125, -66.5)
ax.set_ylim(24, 50)
ax.set_title('US Counties Colored by Cluster Allocation', fontsize=20)
ax.axis('off')

plt.tight_layout()
plt.savefig("us_county_clusters.png", dpi=300, bbox_inches="tight")
plt.show()

 Clearly, there is a geographical component to health (maybe due to pollution). But there also appears to be a political-economic dimension. This illustrates what clustering can achieve! (It may be hard to see, so check out the saved png file.)